### Install required packages

In [ ]:
!pip install openai --quiet

### Import required libraries

In [4]:
from openai import OpenAI
import csv
import time
import pandas as pd
from datetime import datetime
import os
from google.colab import userdata
import json

###Set the API KEY using Google colab user data option

In [5]:
openai_access = userdata.get('open_ai')
os.environ['OPENAI_API_KEY'] = openai_access


### Read the configurations

In [12]:
json_file_path = "/content/drive/MyDrive/Colab Notebooks/config.json"
with open(json_file_path, "r") as file:
    configuration = json.load(file)

### Load the user story descriptions to data frame

In [13]:
input_csv_path = configuration['input_csv_path']
# Read the complete CSV file as a pandas DataFrame
data_frame = pd.read_csv(input_csv_path)


## STEPS
### 1.   Read each user story description
### 2.   Set the model [gpt3.5-turbo, gpt-4] and model parameters
### 3.   Read the zero, few-shot prompt from configuration
### 4.   call OPEN AI chat completion end point
### 5. save the response as .feature file

In [15]:
# Specify the columns to iterate
#step 1
us_desc_column=configuration['us_desc_column']
app_desc_column=configuration['app_desc_column']
# read model, configuration parameters
#step 2
gpt_3_model=configuration['gpt_3.5_model']
gpt_4_model=configuration['gpt_4_model']
gpt_temperature=configuration['gpt_temperature']
gpt_top_p=configuration['gpt_top_p']
gpt_max_tokens=configuration['gpt_max_tokens']
response_file_name_gpt_3=configuration['response_feature_file_gpt_3.5']
response_file_name_gpt_4=configuration['response_feature_file_gpt_4']
few_shot_method=configuration['few_shot_method']
zero_shot_method=configuration['zero_shot_method']
output_csv_path=configuration['output_csv_path']
#step 3
system_prompt=configuration['system_prompt']
openaiclient = OpenAI()
# Iterate over each row to read user story descriptions
for index, row in data_frame.iterrows():
    user_story = row[us_desc_column]
    story_context = row[app_desc_column]
    #step 3
    zero_shot_array=configuration['zero_shot']
    zero_shot=' '.join(zero_shot_array)
    few_shot_user_prompt_array=configuration['few_shot_user_prompt']
    few_shot_user_prompt=' '.join(few_shot_user_prompt_array)
    few_shot_assistant_prompt_array=configuration['few_shot_assistant_prompt']
    few_shot_assistant_prompt=' '.join(few_shot_assistant_prompt_array)
    user_message =   zero_shot+user_story
    #call the openai api for generating feature file by proving zero or few shot prompts
    #step 4
    chat_suggestion = openaiclient.chat.completions.create(
    messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": few_shot_user_prompt},
                {"role":"assistant","content": few_shot_assistant_prompt},
                {"role": "user", "content": user_message},
            ],
            #model=gpt_4_model,
            model=gpt_3_model,
            temperature=gpt_temperature,
            frequency_penalty=0,
            presence_penalty=0,
            top_p=gpt_top_p,
           # max_tokens=gpt_max_tokens,
            #seed=1
        )

    # get the response
    output_response = chat_suggestion.choices[0].message.content
    # create date timestamp
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    response_file_name = f"US{index}_{timestamp}_{few_shot_method}_{response_file_name_gpt_3}.feature"
    # Write the response to a feature file
    # step 5
    feature_file_path = output_csv_path+response_file_name
    with open(feature_file_path, 'w', encoding='utf-8') as feature_file:
      feature_file.write(output_response)
      time.sleep(20)

KeyboardInterrupt: 